In [2]:
import pandas as pd

In [3]:
jobs = pd.read_csv('/content/drive/MyDrive/Báo cáo đồ án chuyên ngành/jobs.tsv', delimiter='\t',encoding='utf-8', error_bad_lines=False)
users = pd.read_csv('/content/drive/MyDrive/Báo cáo đồ án chuyên ngành/users.tsv' ,delimiter='\t',encoding='utf-8')

b'Skipping line 122433: expected 11 fields, saw 12\n'
b'Skipping line 602576: expected 11 fields, saw 12\n'
b'Skipping line 990950: expected 11 fields, saw 12\n'
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
jobs_top_10000 = jobs[:10000]

In [5]:
users_top_10000 = users[:10000]

In [6]:
jobs_top_10000[:2]

,JobID,WindowID,Title,Description,Requirements,City,State,Country,Zip5,StartDate,EndDate
0,1,1,Security Engineer/Technical Lead,<p>Security Clearance Required:&nbsp; Top Secr...,<p>SKILL SET</p>\r<p>&nbsp;</p>\r<p>Network Se...,Washington,DC,US,20531,2012-03-07 13:17:01.643,2012-04-06 23:59:59
1,4,1,SAP Business Analyst / WM,<strong>NO Corp. to Corp resumes&nbsp;are bein...,<p><b>WHAT YOU NEED: </b></p>\r<p>Four year co...,Charlotte,NC,US,28217,2012-03-21 02:03:44.137,2012-04-20 23:59:59


In [7]:
users_top_10000[:2]

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,47,1,Train,Paramount,CA,US,90723,High School,NaN,1999-06-01 00:00:00,3,10.0,Yes,No,0
1,72,1,Train,La Mesa,CA,US,91941,Master's,Anthropology,2011-01-01 00:00:00,10,8.0,Yes,No,0


In [ ]:
users_id = users_top_10000['UserID']

In [ ]:
jobs_id = jobs_top_10000['JobID']

In [ ]:
import random
scores_location = [random.randint(0,3) for i in range(10000)]
scores_location[:10]

[1, 2, 2, 0, 1, 2, 1, 1, 1, 1]

In [ ]:
dict = {'jobs_id': jobs_id, 'users_id': users_id, 'scores_location': scores_location} 
df = pd.DataFrame(dict)

In [ ]:
df.head()

,jobs_id,users_id,scores_location
0,1,47,1
1,4,72,2
2,7,80,2
3,8,98,0
4,9,123,1


# Lọc cộng tác

In [ ]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 10.4 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619385 sha256=71f74ebe83b5cf380240fcc1944fdf9f45a5b7d8327dc4efff66f11ef0b6cbf1
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
from surprise.model_selection import train_test_split

**tạo bảng dòng = users_id, cột = jobsrs_id, giá trị = scores_location**

In [ ]:
scores_matrix = df.pivot_table(values='scores_location', index='users_id', columns='jobs_id', fill_value=0)
scores_matrix.head()

jobs_id,1,4,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,...,36612,36613,36614,36615,36616,36617,36619,36620,36621,36622,36623,36624,36625,36626,36628,36629,36630,36631,36632,36633,36634,36636,36637,36639,36640,36641,36645,36647,36648,36650,36652,36653,36654,36656,36657,36658,36660,36661,36662,36663
users_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
47,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
72,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
80,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
123,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#chuyển vị trí ma trận
X = scores_matrix.T

In [ ]:
X.head()

users_id,47,72,80,98,123,131,162,178,203,344,395,411,415,437,483,496,501,524,534,547,553,554,555,589,640,644,645,655,682,744,755,767,769,861,886,908,951,966,1006,1031,...,205841,205845,205859,205885,205899,205916,205924,205941,205949,205970,205995,206028,206030,206053,206057,206058,206059,206062,206069,206083,206114,206164,206178,206205,206213,206252,206255,206296,206307,206308,206314,206331,206335,206350,206358,206369,206386,206389,206396,206418
jobs_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
X.shape

(10000, 10000)

In [ ]:
#Decomposing the Matrix | phân rã ma trận ( làm ngắn ma trận) Truncated SVDs
from sklearn.decomposition import TruncatedSVD

SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(10000, 10)

In [ ]:
decomposed_matrix[:2]

array([[ 5.05771461e-08, -6.95481764e-08,  2.19316932e-08,
         3.63596106e-08,  7.56628087e-08,  1.25476590e-08,
        -3.69238957e-08, -3.16826762e-08,  4.82210895e-08,
         1.49989092e-08],
       [ 2.92200405e-04, -1.83981401e-04,  3.01049360e-04,
        -3.94002569e-04,  2.88018238e-06, -4.76650039e-04,
        -6.23373345e-05, -1.55170712e-04,  1.22570030e-06,
        -2.91426411e-04]])

In [ ]:
#Correlation Matrix | tạo ma trận tương quan 
import numpy as np
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


(10000, 10000)

**Test thử dự đoán**

In [ ]:
# lấy 1 ID sản phẩm X.index = hàng, X.colums =  cột
X.index[21]

26

In [ ]:
i = 26 # User muốn tìm công việc có jobs_id '26' -> tìm ra ID (số thứ tự) trong list jobs_id

list_jobs_id = list(X.index) # lấy list jobs_id
id = list_jobs_id.index(i) # tìm ID trong list

In [ ]:
id

21

In [ ]:
# tương quan của công việc có id = 21 so với các công việc khác
# tương quan  = 1 chính là giá trị tương quan của chính nó
correlation_by_ID = correlation_matrix[id]
correlation_by_ID

array([ 0.02984754,  0.50162741,  0.5011887 , ..., -0.10475641,
               nan, -0.30878515])

In [ ]:
len(correlation_by_ID)

10000

In [ ]:
# Đề xuất 10 sản phẩmjobs_id có mức độ tương quan cao theo thứ tự
recommend = list(X.index[correlation_by_ID > 0.6])
# xóa công việc user đã tìm (đã tìm thì k cần gợi ý )
recommend.remove(i) 
recommend[0:10]

ValueError: ignored

In [ ]:
def get_recommend_job_by_job(id_job, n_recommend= 10):
  list_jobs_id = list(X.index) # lấy list jobs_id
  id = list_jobs_id.index(i) # tìm ID trong list
  correlation_by_ID = correlation_matrix[id] # lấy ma trận tương quan của job có id_jobs ra

  # Đề xuất n jobs_id có mức độ tương quan cao theo thứ tự
  recommend = list(X.index[correlation_by_ID > 0.6])
  # xóa công việc user đã tìm (đã tìm thì k cần gợi ý )
  recommend.remove(i) 
  return recommend[:n_recommend]

In [ ]:
list_jobs_id[:10]

[1, 4, 7, 8, 9, 10, 11, 12, 13, 14]

In [ ]:
get_recommend_job_by_job(21,5)

[18, 38, 52, 258, 437]

**============================**

In [ ]:
user_a = ['Paramount','CA','US']
job_a = ['Paramount','CA','VN']

In [ ]:
def get_score_location(user_loc, job_loc):
  score = 0
  for i in range(0,3):
    if(user_a[i] == job_a[i]):
      score = score + 1
  return score


In [ ]:
users_location = users[['City','State','Country']]
jobs_location = jobs[['City','State','Country']]

In [ ]:
user_l = []
for index, row in users_location.iterrows():
    user_temp = []
    user_temp.append(row["City"])
    user_temp.append(row["State"])
    user_temp.append(row["Country"])
    user_l.append(user_temp)

In [ ]:
job_l = []
for index, row in users_location.iterrows():
    job_temp = []
    job_temp.append(row["City"])
    job_temp.append(row["State"])
    job_temp.append(row["Country"])
    job_l.append(user_temp)

In [ ]:
score_l = []
for i in range(1000):
  score = get_score_location(user_l[i],job_l[i])
  score_l.append(score)

In [ ]:
score_l[100:1000]

[2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
